In [41]:
import http.client, urllib.request, urllib.parse, urllib.error, base64

headers = {
    # Request headers
    'Content-Type': 'application/json',
    'Ocp-Apim-Subscription-Key': '68ca10eb69f34212b22956e6b9f658d0',
}

params = urllib.parse.urlencode({
})

0.032561659812927246


In [125]:
%store -r data
data.head()

,sentiment,text
1,positive,Greetings and welcome to the Microsoft Fiscal ...
10,positive,technology. Microsoft 365 helps every organiza...
100,positive,is coming from the line of Walter Pritchard wi...
101,neutral,we're growing at eye-popping rates right now. ...
102,positive,the key things that we think about is differen...


In [126]:
texts = data.text
labels = data.sentiment.map(str.lower).map({'positive' : 1, 'neutral' : 0, 'negative' : -1}).map(int)

In [49]:
pred_labels = []

In [109]:
for text in texts:
    try:
        conn = http.client.HTTPSConnection('eastus.api.cognitive.microsoft.com')
        conn.request("POST", "/text/analytics/v2.0/sentiment?%s" % params, '{"documents": [{"id": "1","text": "%s"}' %text, headers)
        response = conn.getresponse()
        data = json.loads(response.read().decode(response.headers.get_content_charset('utf-8')))
        data = data['documents'][0]
        pred_labels.append(data['score'])
        conn.close()
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))

In [114]:
scores_new = pred_labels

In [115]:
for i in range(len(pred_labels)):
    if pred_labels[i] < 0.5:
        scores_new[i]= -1
    elif pred_labels[i] > 0.5:
        scores_new[i] = 1
    else:
        scores_new[i] = 0

In [129]:
labels_azure = scores_new
%store labels_azure
del labels_azure

Stored 'labels_azure' (list)


In [118]:
from sklearn.metrics import confusion_matrix

In [128]:
confusion_matrix(y_pred=scores_new, y_true=labels)

array([[  1,  35,  26],
       [  4, 128, 111],
       [ 11, 166, 140]], dtype=int64)